# Data Profiling
> Generate a html data profiler

In [ ]:
#| default_exp etl.national.data_profiling

: 

In [ ]:
#| export
import pandas as pd 

import sweetviz as sv

from pathlib import Path
from os import listdir, path, mkdir
from nilo_iq import tables

: 

In [ ]:
#| export

DATA_DIR = path.join(tables.ROOT_DIRECTORY,'data/NationalPrices')
PROFILING_DIR = path.join(tables.ROOT_DIRECTORY,'sweet_viz/NationalPrices')

def list_and_create_nested_dir(
    data_files: list,
    profile_files: list,
    _dir: str ='',
):
    
    _listdir = listdir(path.join(DATA_DIR, _dir))

    for _name in _listdir:

        if '.gitignore' in _name or '.dvc' in _name:
            continue

        _abs_data_name = path.join(DATA_DIR, _dir, _name)
        _abs_profile_name = path.join(PROFILING_DIR, _dir, _name)

        if path.isfile(_abs_data_name):
            data_files.append(_abs_data_name)
            profile_files.append(_abs_profile_name)
            continue

        if not path.isdir(_abs_profile_name):
            mkdir(_abs_profile_name)

        list_and_create_nested_dir(
            data_files=data_files,
            profile_files=profile_files,
            _dir = path.join(_dir, _name),
        )


data_files = []
profile_files = []

list_and_create_nested_dir(
    data_files=data_files,
    profile_files=profile_files,
)

: 

In [ ]:
#| export



rewrite_profile = False
for k,_ in enumerate(data_files):

    _profile_file =  profile_files[k]+'.html'
    if not rewrite_profile and path.isfile(_profile_file):
        continue
        
    
    df = pd.read_csv(data_files[k])

    if 'precio_minimo' in df.columns:
        df['precio_minimo'] = pd.to_numeric(df['precio_minimo'], errors='coerce')
        df['precio_maximo'] = pd.to_numeric(df['precio_maximo'], errors='coerce')
        
    my_report = sv.analyze(df)

    my_report.show_html(    
        filepath=_profile_file,
        open_browser=False,
        layout='widescreen',
        scale=None,)

: 

: 